In [1]:
import pandas as pd
import numpy as np

np.random.seed(42)

In [2]:
def generate_instance():
    # iç veri
    ay = np.max([1.0, np.round(np.random.normal(3, 1))]) if np.random.rand() > 0.93 else np.max([1.0, np.round(np.random.normal(28, 7))])
    kredi_karti_aldi = 1.0 if np.random.rand() > 0.99 else 0.0
    kredi_karti_sahibi = 1.0 if np.random.rand() > 0.7 else 0.0
    kredi_karti_ilk_ay_harcama = 0.0 if (kredi_karti_sahibi == 0.0 and kredi_karti_aldi == 0.0) else np.max([0.0, np.random.normal(1000.0, 250.0)])
    kredi_karti_harcama = 0.0 if (kredi_karti_sahibi == 0.0 and kredi_karti_aldi == 0.0) else np.max([0.0, np.random.normal(3000, 2000)])
    onceki_ay_kredi_karti_harcama = 0.0 if kredi_karti_sahibi == 0.0 else np.max([0.0, np.random.normal(3000, 2000)])
    toplam_varlik = 0.0 if np.random.rand() > 0.95 else np.max([0.0, np.random.normal(20000, 8000)])
    toplam_varlik_gecen_ay = 0.0 if ay < 2 or np.random.rand() > 0.5 else np.max([0.0, np.random.normal(20000, 8000)])
    toplam_varlik_onceki_3_ay_ortalama = 0.0 if ay < 4 else np.max([toplam_varlik_gecen_ay, np.random.normal(20000, 5000)])
    acik_kredi = 1.0 if np.random.rand() > 0.8 else 0.0
    onceki_ay_urun_sayisi = 0.0 if np.random.rand() > 0.995 else np.max([0.0, np.round(np.random.normal(4.0, 1.5))])
    urun_sayisi = 0.0 if np.random.rand() > 0.98 else np.max([0.0, np.round(np.random.normal(4.0, 1.5))])
    
    # performans ölçütleri
    yeni_musteri = 1.0 if ay <= 3.0 else 0.0
    capraz_satis = 1.0 if ay > 3.0 and (kredi_karti_sahibi == 1.0 and kredi_karti_ilk_ay_harcama > 500.0) and np.random.rand() > 0.8 else 0.0
    yukari_satis = 1.0 if ay > 3.0 and (toplam_varlik_gecen_ay > 10000 and toplam_varlik / toplam_varlik_onceki_3_ay_ortalama > 1.5) else 0.0
    
    devam_eden_musteri = 0.0
    terk = 0.0
    geri_kazanilmis_musteri = 0.0
    pasif_musteri = 0.0
    aktiflestirilmis_musteri = 0.0
    
    if ay > 3 and kredi_karti_sahibi == 0 and toplam_varlik == 0 and acik_kredi == 0:
        pasif_musteri = 1.0
    elif ay > 3 and urun_sayisi == 0:
        terk = 1.0
    elif ay > 3 and onceki_ay_urun_sayisi == 0.0 and urun_sayisi > 0:
        geri_kazanilmis_musteri = 1.0
    elif ay > 3 and toplam_varlik_gecen_ay == 0.0 and onceki_ay_kredi_karti_harcama == 0.0 and onceki_ay_urun_sayisi > 0 and urun_sayisi > 0 and toplam_varlik != 0 and toplam_varlik_onceki_3_ay_ortalama != 0 and np.random.rand() > 0.9:
        aktiflestirilmis_musteri = 1.0
    elif ay > 3:
        devam_eden_musteri = 1.0
        
    tavsiye = np.random.normal(6.5, 0.25) # bias
    tavsiye += yeni_musteri * np.random.normal(2, 0.2)
    tavsiye += capraz_satis * np.random.normal(1, 0.1)
    tavsiye += yukari_satis * np.random.normal(3, 0.001)
    tavsiye += devam_eden_musteri * np.random.normal(0.5, 0.25)
    tavsiye += terk * np.random.normal(-3.0, 0.002)
    tavsiye += geri_kazanilmis_musteri * np.random.normal(1.0, 0.5)
    tavsiye += aktiflestirilmis_musteri * np.random.normal(0.5, 0.25)
    tavsiye += pasif_musteri * np.random.normal(-1.5, 0.1)
    tavsiye += np.random.normal(0.5, 0.25)
    tavsiye = np.round(tavsiye)
    tavsiye = np.max([0.0, tavsiye])
    tavsiye = np.min([10.0, tavsiye])
    tavsiye = tavsiye * 0.1
    
    memnuniyet = np.random.normal(30.0, 0.5)
    memnuniyet += ay * np.random.normal(0.5, 0.01)
    memnuniyet += (urun_sayisi - 4) * np.random.normal(1.0, 0.01)
    memnuniyet += (toplam_varlik - 5000) * np.random.normal(0.001, 0.0001)
    memnuniyet += (kredi_karti_sahibi - 1) * np.random.normal(5.0, 0.5)
    memnuniyet += np.random.normal(10.0, 0.5)
    memnuniyet = np.round(memnuniyet)
    memnuniyet = np.max([0.0, memnuniyet])
    memnuniyet = np.min([100.0, memnuniyet])
    memnuniyet = memnuniyet * 0.01
    
    if tavsiye > 0.7 and np.random.rand() > 0.9:
        capraz_satis = 1.0
    if tavsiye > 0.8 and memnuniyet > 0.8 and np.random.rand() > 0.9:
        yukaris_satis = 1.0
    
    
    # digital
    basarili_giris_sayisi = 0.0
    gezdigi_sayfa_sayisi = 0.0
    hata_ile_sonlanan_islem_sayisi = 0.0
    bir_islem_icin_ortalama_tiklama_sayisi = 0.0
    cevrim_ici_sure = 0.0
    
    # classic
    atm_kullanim = 0.0
    sube_kullanim = 0.0
    sube_sure = 0.0
    
    if yeni_musteri == 1.0:
        basarili_giris_sayisi = np.max([0.0, np.round(np.random.normal(4, 0.5) * 0.1)])
        gezdigi_sayfa_sayisi = np.max([0.0, np.round(np.random.normal(1, 0.5))])
        hata_ile_sonlanan_islem_sayisi = np.max([0.0, np.round(np.random.normal(0.8, 0.1))])
        bir_islem_icin_ortalama_tiklama_sayisi = np.max([0.0, np.round(np.random.normal(6, 0.5) * 0.1)])
        cevrim_ici_sure = np.max([0.0, np.random.normal(6, 0.5) * 0.1])
        atm_kullanim = np.max([0.0, np.round(np.random.normal(1, 0.1))])
        sube_kullanim = np.max([0.0, np.round(np.random.normal(1, 0.2))])
        sube_sure = np.max([0.0, np.random.normal(0.5, 0.3)])
        
    elif devam_eden_musteri == 1.0:
        basarili_giris_sayisi = np.max([0.0, np.round(np.random.normal(8, 0.25) * 0.1)])
        gezdigi_sayfa_sayisi = np.max([0.0, np.round(np.random.normal(3, 0.5))])
        hata_ile_sonlanan_islem_sayisi = np.max([0.0, np.round(np.random.normal(1.1, 0.2))])
        bir_islem_icin_ortalama_tiklama_sayisi = np.max([0.0, np.round(np.random.normal(4, 0.5) * 0.1)])
        cevrim_ici_sure = np.max([0.0, np.random.normal(10, 0.5) * 0.1])
        atm_kullanim = np.max([0.0, np.round(np.random.normal(3, 0.1))])
        sube_kullanim = np.max([0.0, np.round(np.random.normal(2, 0.2))])
        sube_sure = np.max([0.0, np.random.normal(1.0, 0.3)])
    elif terk == 1.0:
        basarili_giris_sayisi = np.max([0.0, np.round(np.random.normal(0.5, 0.2) * 0.1)])
        gezdigi_sayfa_sayisi = np.max([0.0, np.round(np.random.normal(3.0, 0.3))])
        hata_ile_sonlanan_islem_sayisi = np.max([0.0, np.round(np.random.normal(1.0, 0.5))])
        bir_islem_icin_ortalama_tiklama_sayisi = np.max([0.0, np.round(np.random.normal(9, 0.8) * 0.1)])
        cevrim_ici_sure = np.max([0.0, np.random.normal(4, 0.5) * 0.1])
        atm_kullanim = np.max([0.0, np.round(np.random.normal(2, 0.1))])
        sube_kullanim = np.max([0.0, np.round(np.random.normal(1, 0.5))])
        sube_sure = np.max([0.0, np.random.normal(3.0, 0.5)])
    elif geri_kazanilmis_musteri == 1.0:
        basarili_giris_sayisi = np.max([0.0, np.round(np.random.normal(3, 0.1) * 0.1)])
        gezdigi_sayfa_sayisi = np.max([0.0, np.round(np.random.normal(1.5, 0.3))])
        hata_ile_sonlanan_islem_sayisi = np.max([0.0, np.round(np.random.normal(0.2, 0.1))])
        bir_islem_icin_ortalama_tiklama_sayisi = np.max([0.0, np.round(np.random.normal(6, 0.5) * 0.1)])
        cevrim_ici_sure = np.max([0.0, np.random.normal(4, 0.2) * 0.1])
        atm_kullanim = np.max([0.0, np.round(np.random.normal(1, 0.1))])
        sube_kullanim = np.max([0.0, np.round(np.random.normal(1.5, 0.1))])
        sube_sure = np.max([0.0, np.random.normal(0.25, 0.3)])
    elif aktiflestirilmis_musteri == 1.0:
        basarili_giris_sayisi = np.max([0.0, np.round(np.random.normal(4, 0.2) * 0.1)])
        gezdigi_sayfa_sayisi = np.max([0.0, np.round(np.random.normal(1, 0.5))])
        hata_ile_sonlanan_islem_sayisi = np.max([0.0, np.round(np.random.normal(0.1, 0.1))])
        bir_islem_icin_ortalama_tiklama_sayisi = np.max([0.0, np.round(np.random.normal(5, 0.5) * 0.1)])
        cevrim_ici_sure = np.max([0.0, np.random.normal(5, 0.5) * 0.1])
        atm_kullanim = np.max([0.0, np.round(np.random.normal(2, 0.1))])
        sube_kullanim = np.max([0.0, np.round(np.random.normal(0.5, 0.1))])
        sube_sure = np.max([0.0, np.random.normal(0.25, 0.2)])
    
    instance = [ay, kredi_karti_aldi, kredi_karti_sahibi, kredi_karti_ilk_ay_harcama, kredi_karti_harcama, onceki_ay_kredi_karti_harcama, toplam_varlik, toplam_varlik_gecen_ay, toplam_varlik_onceki_3_ay_ortalama,
    acik_kredi, onceki_ay_urun_sayisi, urun_sayisi, yeni_musteri, capraz_satis, yukari_satis, devam_eden_musteri, terk, geri_kazanilmis_musteri, pasif_musteri, aktiflestirilmis_musteri,
    tavsiye, basarili_giris_sayisi, gezdigi_sayfa_sayisi, hata_ile_sonlanan_islem_sayisi, bir_islem_icin_ortalama_tiklama_sayisi, cevrim_ici_sure, atm_kullanim, sube_kullanim, sube_sure, memnuniyet]
    
    return np.array(instance)
    
    
    
    

In [3]:
samples = np.array([generate_instance() for x in range(100000)])
columns = ["ay", "kredi_karti_aldi", "kredi_karti_sahibi", "kredi_karti_ilk_ay_harcama", "kredi_karti_harcama", "onceki_ay_kredi_karti_harcama", "toplam_varlik", "toplam_varlik_gecen_ay", "toplam_varlik_onceki_3_ay_ortalama",
    "acik_kredi", "onceki_ay_urun_sayisi", "urun_sayisi", "yeni_musteri", "capraz_satis", "yukari_satis", "devam_eden_musteri", "terk", "geri_kazanilmis_musteri", "pasif_musteri", "aktiflestirilmis_musteri",
    "tavsiye", "basarili_giris_sayisi", "gezdigi_sayfa_sayisi", "hata_ile_sonlanan_islem_sayisi", "bir_islem_icin_ortalama_tiklama_sayisi", "cevrim_ici_sure", "atm_kullanim", "sube_kullanim", "sube_sure", "memnuniyet"]
data = pd.DataFrame(samples, columns = columns)

In [4]:
data["yeni_musteri"].sum() / data.shape[0]

0.04844

In [5]:
data["capraz_satis"].sum() / data.shape[0]

0.14318

In [6]:
data["yukari_satis"].sum() / data.shape[0]

0.02746

In [7]:
data["devam_eden_musteri"].sum() / data.shape[0]

0.85007

In [8]:
data["terk"].sum() / data.shape[0]

0.02808

In [9]:
data["geri_kazanilmis_musteri"].sum() / data.shape[0]

0.01336

In [10]:
data["pasif_musteri"].sum() / data.shape[0]

0.0291

In [11]:
data["aktiflestirilmis_musteri"].sum() / data.shape[0]

0.03095

In [12]:
data["tavsiye"].mean()

0.7542150000000561

In [13]:
data["memnuniyet"].mean()

0.6354270000000133

In [14]:
data["sube_kullanim"].mean()

1.81154

In [15]:
data.var()

ay                                        8.655082e+01
kredi_karti_aldi                          9.753075e-03
kredi_karti_sahibi                        2.086387e-01
kredi_karti_ilk_ay_harcama                2.301333e+05
kredi_karti_harcama                       3.025880e+06
onceki_ay_kredi_karti_harcama             3.032908e+06
toplam_varlik                             7.879434e+07
toplam_varlik_gecen_ay                    1.317985e+08
toplam_varlik_onceki_3_ay_ortalama        5.177402e+07
acik_kredi                                1.601695e-01
onceki_ay_urun_sayisi                     2.372079e+00
urun_sayisi                               2.597472e+00
yeni_musteri                              4.609403e-02
capraz_satis                              1.226807e-01
yukari_satis                              2.670622e-02
devam_eden_musteri                        1.274523e-01
terk                                      2.729179e-02
geri_kazanilmis_musteri                   1.318164e-02
pasif_must

In [16]:
#data=(data-data.min())/(data.max()-data.min())

In [17]:
data.to_csv("../data/generated.csv", sep="\t", index=False)

In [18]:
with_pasif = ["tavsiye", "yeni_musteri", "capraz_satis", "yukari_satis", "devam_eden_musteri", "terk", "geri_kazanilmis_musteri", "pasif_musteri", "aktiflestirilmis_musteri"]
data[with_pasif].to_csv("../data/gen.csv", sep="\t", index=False)

In [19]:
without_pasif = ["tavsiye", "yeni_musteri", "capraz_satis", "yukari_satis", "devam_eden_musteri", "terk", "geri_kazanilmis_musteri", "aktiflestirilmis_musteri"]
data[without_pasif].to_csv("../data/gen2.csv", sep="\t", index=False)

In [20]:
with_digital = ["tavsiye", "yeni_musteri", "capraz_satis", "yukari_satis", "devam_eden_musteri", "terk", "geri_kazanilmis_musteri", "aktiflestirilmis_musteri", "basarili_giris_sayisi", "gezdigi_sayfa_sayisi", "hata_ile_sonlanan_islem_sayisi", "bir_islem_icin_ortalama_tiklama_sayisi", "cevrim_ici_sure", "atm_kullanim", "sube_kullanim", "sube_sure"]
data[with_digital].to_csv("../data/gen_digital.csv", sep="\t", index=False)